## k近傍 k-Nearest Neighbor; k-NN
$d$次元ベクトル空間内においてある未知のデータ点に対して既知のデータ点の中から（主にユークリッド距離で）近傍の$k$個の点を選出する。

### データの用意
まず、ベクトル空間の次元`d`を定義する。次に既知のデータ点を格納したデータベース配列と検索対象のデータ点を格納したデータ配列を定義する。

![IndexFlatL2](./img/IndexFlatL2.png)

In [1]:
# prepare data
import time
import numpy as np
import faiss
d = 64                         # ベクトルの次元(dimension)
nb = 1000000                    # データベースのサイズ(database size)
nq = 100000                     # クエリベクトルの数(nb of queries)
np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

## Index配列

### IndexFlatL2
L2ノルムを近さの基準とした完全探索を行う配列。([他の基準](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes))

```python
import faiss
cpu_index = faiss.IndexFlatL2(d) # cpu使用時
index = faiss.GpuIndexFlatL2(res, d, flat_config)
```

params:
- `res`: `faiss.StandardGpuResources`
- `d`: `int`, ベクトルの次元, dimension of each vector
- `flat_config`: `faiss.GpuIndexFlatConfig`

instance:
- `.is_trained`: `bool=True`, 訓練モードかどうか(whether training is required)（`IndexFlatL2`では関係ない）
- `.ntotal`: `int`,  ベクトルの数(number of vectors)
- `.add(x)`:  ベクトルを追加する(append vectors to the index)
  - params:
     - `x`: `np.ndarray`, `nb`個のベクトルを格納した配列(database of vectors), `shape = (nb, d)`
- `.search(q, k)`: クエリベクトル`q[i]`に近い`k`個のベクトルを返す(return vectors k nearest neighbors in the database)
  - params:
    - `q`: `np.ndarray`, `nq`個のクエリベクトルを格納した配列(query vectors), `shape=(nq, d)`
    - `k`: `int`, 返す近傍点の数(number of return neighbors)
  - return:
    - `D`: `np.ndarray`, 各近傍点のクエリからの距離, `shape=(nq, k)`
    - `I`: `np.ndarray`, 各近傍点のインデックス, `shape=(nq, k)`
![search return](./img/search_returnDI.png)

#### GPU使用時のパラメータを得るには
```python
res = faiss.StandardGpuResources() # resource
```

return:
- `faiss.StandardGpuResources`, GPUリソース(GPU resources)

```python
flat_config = faiss.GpuIndexFlatConfig()
flat_config.useFloat16 = False
flat_config.device = i
```

instance:
- `useFloat16`: `bool=False`, 16bitか32bitか(whether 16 or 32bits)
- `device`: `int=0`, GPUのID(gpu id)

return:
- `faiss.GpuIndexFlatConfig`, GPUの設定(GPU config) 

In [2]:
# GpuIndexFlatL2オブジェクトを作成
print("BUILD THE INDEX")
res = faiss.StandardGpuResources()
flat_config = faiss.GpuIndexFlatConfig()
index = faiss.GpuIndexFlatL2(res, d, flat_config) 

print(index.is_trained)

#ベクトルを追加(append vectors to the index)
index.add(xb)
print(index.ntotal)

print("SEARCH KNN")
k = 4 # 近傍点の個数kを決定(we want to see k nearest neighbors)

# 動作確認(sanity check) -> Iの1列目が0, 1, 2, 3, 4、Dの1列目が0か0に近い値であるなら正常
D, I = index.search(xb[:5], k)
print(I)
print(D)

# 探索実行(search)
s = time.time()
D, I = index.search(xq, k)
e = time.time()
print(I[-5:])
print(D[-5:])
print("time: {}".format(e-s))

BUILD THE INDEX
True
1000000
SEARCH KNN
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[-3.8146973e-06  7.1751747e+00  7.2076283e+00  7.2511635e+00]
 [ 0.0000000e+00  6.3235664e+00  6.6845760e+00  6.7999439e+00]
 [-3.8146973e-06  5.7964096e+00  6.3917313e+00  7.2815094e+00]
 [ 0.0000000e+00  7.2779083e+00  7.5279884e+00  7.6628475e+00]
 [ 0.0000000e+00  6.7638016e+00  7.2951202e+00  7.3688164e+00]]
[[ 99270 100884  99779 100369]
 [ 99676  99024  99952  99431]
 [100445 100135  99779  99899]
 [100429  99859  99039 100219]
 [ 99691 100457 100337  99105]]
[[7.069336  7.2216797 7.2304688 7.288086 ]
 [5.9853516 6.0634766 6.2578125 6.4414062]
 [7.2841797 7.3164062 7.493164  7.5976562]
 [6.6367188 7.2001953 7.213867  7.260742 ]
 [5.458008  5.864258  6.4208984 6.4296875]]
time: 8.5036780834198


## 高速化
### IndexIVFFlat
`metric`で指定した基準に基づいて精度低下と引き換えに高速探索を行う。

```python
index = faiss.GpuIndexIVFFlat(res, d, nlist, metric, ivfflat_config)
```

params:
- `res`: `faiss.StandardGpuResources`
- `d`: `int`, ベクトルの次元(dimension of each vector)
- `nlist`: `int`, ボロノイ領域の数(number of Voronoi cells)
- `metric`: `int`, 距離基準(metric to search)
    - `faiss.METRIC_INNER_PRODUCT=0`: コサイン類似度(デフォルト)
    - `faiss.METRIC_L2=1`: L2ノルム
- `ivfflat_config`: `faiss.GpuIndexIVFFlatConfig`

instance:
- `.is_trained`: `bool=True`, 訓練モードかどうか(whether training is required)
- `.train(x)`: ボロノイ領域を学習する(train Voronoi cells)
  - params:
     - `x`: `np.ndarray`, `nb`個のベクトルを格納した配列(database of vectors), `shape = (nb, d)`
- `.add(x)`:  ベクトルを追加する(append vectors to the index)
- `.setNumProbes(nprobe)`: 検索するボロノイ領域の個数`nprobe`をセットする(set `nprobe`, the number of cells that are visited to perform a search)
  - params:
     - `n`: `int`, 検索するボロノイ領域の個数(the number of cells that are visited to perform a search)
- `.search(q, k)`: クエリベクトル`q[i]`に近い`k`個のベクトルを返す(return vectors k nearest neighbors in the database)




In [3]:
nlist = 100
k = 4
res = faiss.StandardGpuResources()
ivfflat_config = faiss.GpuIndexIVFFlatConfig()
index = faiss.GpuIndexIVFFlat(res, d, nlist, faiss.METRIC_L2, ivfflat_config)
       # here we specify METRIC_L2, by default it performs inner-product search

assert not index.is_trained
s = time.time()
index.train(xb)
e = time.time()
print("train time: {}".format(e-s))
assert index.is_trained

index.add(xb)                  # add may be a bit slower as well
s = time.time()
index.setNumProbes(1)
D, I1 = index.search(xq, k)     # actual search
e = time.time()
print(I1[-5:])                  # neighbors of the 5 last queries
print("search time: {}".format(e-s))
print("accuracy: {:.1f}%".format(np.sum(I==I1)/(k*nq)*100))

s = time.time()
index.setNumProbes(10)         # default nprobe is 1, try a few more
D, I2 = index.search(xq, k)
e = time.time()
print(I2[-5:])                  # neighbors of the 5 last queries
print("search time: {}".format(e-s))
print("accuracy: {:.1f}%".format(np.sum(I==I2)/(k*nq)*100))

train time: 0.18445420265197754
[[ 99270 100884  99779 100369]
 [ 99676  99024  99952  99431]
 [100445 100135  99779  99899]
 [100429  99859  99039  99778]
 [ 99691 100457 100337  99105]]
search time: 1.5525236129760742
accuracy: 93.6%
[[ 99270 100884  99779 100369]
 [ 99676  99024  99952  99431]
 [100445 100135  99779  99899]
 [100429  99859  99039  99778]
 [ 99691 100457 100337  99105]]
search time: 17.26750946044922
accuracy: 99.2%
